# Random Forest Regression

## Importowanie bibliotek

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importowanie zbiorów danych

In [5]:
train_dataset = pd.read_csv('train_dataset.csv')

X = train_dataset.drop(columns=['Cena']).values
y = train_dataset['Cena'].values

## Podział zbioru danych na zbiór treningowy i testowy

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## Skalowanie cech

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Training the Random Forest Regression model on the whole dataset

### Model 1 - sprawdzenie koncepcji

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(
    n_estimators=50,          # Mniej drzew dla szybszego trenowania
    max_depth=20,             # Ograniczona głębokość, aby uniknąć przeuczenia
    max_features='sqrt',      # Pierwiastek z liczby cech (~34 na 1200)
    min_samples_split=10,     # Większa minimalna liczba próbek do podziału, aby uniknąć nadmiernego dopasowania
    min_samples_leaf=5,       # Minimum 5 próbek w liściu, by zmniejszyć overfitting
    n_jobs=-1,                # Wykorzystanie wszystkich rdzeni procesora
    random_state=42
)

regressor.fit(X_train, y_train)

### Model 2 - unikanie przeuczenia

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(
    n_estimators=100,         # Mniej drzew dla szybszego trenowania, więcej dla redukcji wariancji
    max_depth=15,             # Ograniczona głębokość, aby uniknąć przeuczenia lub zwiększona, aby lepiej uchwycić zależności
    max_features='sqrt',      # Pierwiastek z liczby cech (~34 dla 1200)
    min_samples_split=15,     # Większa minimalna liczba próbek do podziału, aby uniknąć nadmiernego dopasowania lub mniejsza dla sklomplikowanych podziałów
    min_samples_leaf=10,      # Minimum 5 próbek w liściu, by zmniejszyć overfitting
    bootstrap=True,           # Losowe podpróbki dla lepszej stabilności
    n_jobs=-1,                # Wykorzystanie wszystkich rdzeni procesora
    random_state=42
)

regressor.fit(X_train, y_train)

### Model 3 - mocne dopasowanie do wzorców

In [ ]:
regressor = RandomForestRegressor(
    n_estimators=300,       # Więcej drzew dla redukcji wariancji
    max_depth=30,           # Głębsze drzewa, aby lepiej uchwycić zależności
    max_features='sqrt',    # Standardowa wartość (~34 na 1200)
    min_samples_split=5,    # Mniejsza wartość dla bardziej skomplikowanych podziałów
    min_samples_leaf=2,     # Bardziej szczegółowe liście
    bootstrap=True,         # Podpróbki dla stabilności
    n_jobs=-1, 
    random_state=42
)

regressor.fit(X_train, y_train)

### Model 4 - podkręcony model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(
    n_estimators=1000,       # Więcej drzew dla redukcji wariancji
    max_depth=50,           # Głębsze drzewa, aby lepiej uchwycić zależności
    max_features='sqrt',    # Standardowa wartość (~34 na 1200)
    min_samples_split=5,    # Mniejsza wartość dla bardziej skomplikowanych podziałów
    min_samples_leaf=2,     # Bardziej szczegółowe liście
    bootstrap=False,         # Podpróbki dla stabilności
    n_jobs=-1, 
    random_state=42
)

regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=100, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=5,
                      n_estimators=1000, n_jobs=-1, random_state=42)

## Przewidywanie wartości zbioru testowego

In [23]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)) # sposób na zmianę rozmiarów wektora

# axix 0 vertical, 1 horizontal

[[ 23365.02   6800.  ]
 [ 65374.42  39900.  ]
 [ 21146.32   9200.  ]
 ...
 [498445.85 555555.  ]
 [ 55638.5   65900.  ]
 [ 30235.6   25850.  ]]


## RMSE

In [24]:
from sklearn.metrics import mean_squared_error

# Obliczenie RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Wyświetlenie wyniku
print(f"RMSE: {rmse:.2f}")

RMSE: 34945.13


## Eliminacja cech - tu niepotrzebna, nie zwiększa dokładności

In [ ]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)


In [ ]:
# Pobranie ważności cech
importances = regressor.feature_importances_

# Tworzymy DataFrame z cechami i ich ważnością
feature_importance_df = pd.DataFrame({'Feature': X_train_df.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Wybieramy tylko cechy, które mają np. co najmniej 0.01 ważności
threshold = 0.01
selected_features = feature_importance_df[feature_importance_df['Importance'] > threshold]['Feature'].tolist()

print("Wybrane cechy:", selected_features)

In [ ]:
to_drop = [col for col in X_train_df.columns if col not in selected_features]
X_train_reduced = X_train_df.drop(columns=to_drop)
X_test_reduced = X_test_df.drop(columns=to_drop)

# Wypisujemy usunięte cechy oraz nowy zbiór cech
print("Usunięte cechy:", to_drop)
print("Nowy zbiór cech:", X_train_reduced.columns)
print("Liczba wybranych cech:", len(X_train_reduced.columns))

## Trenowanie modelu po eliminacji cech

In [ ]:
regressor.fit(X_train_reduced, y_train)

### Przewidywanie na zbiorze testowym

In [ ]:
y_pred = regressor.predict(X_test_reduced)

In [ ]:
# Ustawienie precyzji wyświetlania
np.set_printoptions(precision=0)

# Połączenie przewidywań (y_pred) z rzeczywistymi wartościami (y_test)
result = np.concatenate((y_pred.reshape(-1, 1), y_test.reshape(-1, 1)), axis=1)

# Wypisanie wyników
print(result)

## RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

# Obliczenie RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Wyświetlenie wyniku
print(f"RMSE: {rmse:.2f}")

# Przygotowanie wyników dla zbioru testowego (bez podanych cen)

In [12]:
final_test_dataset = pd.read_csv('test_dataset.csv')

In [13]:
final_test_dataset.shape

(72907, 1351)

In [14]:
train_dataset.head()

,ID,Cena,Stan,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Emisja_CO2,Liczba_drzwi,Rok_publikacji,...,Kolor_burgundy,Kolor_golden,Kolor_gray,Kolor_green,Kolor_other,Kolor_red,Kolor_silver,Kolor_violet,Kolor_white,Kolor_yellow
0,1.0,13900.0,1.0,2005.0,213000.0,170.0,1998.0,232.0,5.0,2021.0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,25900.0,1.0,2010.0,117089.0,110.0,1598.0,159.0,5.0,2021.0,...,0,0,0,0,0,0,1,0,0,0
2,3.0,35900.0,1.0,2015.0,115600.0,136.0,1598.0,145.0,5.0,2021.0,...,0,0,0,0,0,0,0,0,1,0
3,4.0,5999.0,1.0,2007.0,218000.0,90.0,1560.0,118.0,5.0,2021.0,...,0,0,0,0,0,0,0,0,0,0
4,5.0,44800.0,1.0,2013.0,174645.0,142.0,1798.0,152.0,4.0,2021.0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
final_test_dataset.insert(1, 'Cena', 0)
final_test_dataset.head()

,ID,Cena,Stan,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,Emisja_CO2,Liczba_drzwi,Rok_publikacji,...,Kolor_burgundy,Kolor_golden,Kolor_gray,Kolor_green,Kolor_other,Kolor_red,Kolor_silver,Kolor_violet,Kolor_white,Kolor_yellow
0,1.0,0,0.0,2021.0,1.0,163.0,1950.0,153.0,5.0,2021.0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,0,1.0,2008.0,202585.0,145.0,1999.0,184.0,5.0,2021.0,...,0,0,0,0,1,0,0,0,0,0
2,3.0,0,1.0,2005.0,373000.0,218.0,2993.0,155.0,5.0,2021.0,...,0,0,1,0,0,0,0,0,0,0
3,4.0,0,0.0,2021.0,10.0,130.0,1498.0,133.0,5.0,2021.0,...,0,0,0,0,0,0,0,0,1,0
4,5.0,0,1.0,2014.0,150000.0,245.0,2967.0,156.0,5.0,2021.0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
X_final = final_test_dataset.drop(columns=['Cena'])

In [17]:
X_final = sc.transform(X_final)

/Users/mieszkokozikowski/Documents/Conda/miniconda3/envs/radenv_2/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [19]:
y_final_pred = regressor.predict(X_final)

y_final_pred = np.round(y_final_pred).astype(int)

print(y_final_pred.reshape(-1, 1))

[[228073]
 [ 20925]
 [ 39496]
 ...
 [ 57463]
 [ 24132]
 [153081]]


In [20]:
# Tworzenie DataFrame z ID i przewidywanymi cenami
submission = pd.DataFrame({
    'ID': final_test_dataset['ID'].astype('Int32'),  
    'Cena': y_final_pred  
})

In [21]:
# Zapis do pliku
submission.to_csv('predicted_prices.csv', index=False)

print("Predykcje zostały zapisane do 'predicted_prices.csv'")

Predykcje zostały zapisane do 'predicted_prices.csv'
